# Get all Japanese researchers

In [1]:
### read grid file

# 数値計算ライブラリ
# import numpy as np
import pandas as pd

grid_file = "GRID_alljapan.xlsx"
grid_file="GRID_skipped.xlsx"
griddf = pd.read_excel(grid_file)

In [2]:
griddf.head()

,grid_id,name,研究機関名（和名）
0,grid.258799.8,Kyoto University,京都大学
1,grid.177174.3,Kyushu University,九州大学
2,grid.136593.b,Osaka University,大阪大学
3,grid.27476.30,Nagoya University,名古屋大学
4,grid.26999.3d,University of Tokyo,東京大学


In [3]:
import json 
import pandas as pd
from pandas.io.json import json_normalize
from tqdm import tnrange, tqdm_notebook as bar
from time import sleep
import dimcli 
from dimcli.shortcuts import dslquery, dslqueryall
dimcli.login(username="noritaka.usami.a3n@cao.go.jp", password="CO287180")
dsl = dimcli.Dsl()

DimCli v0.6.1 - Succesfully connected to <https://app.dimensions.ai> (method: manual login)


In [5]:
# get researcher ids for a given grid id and publication year

start_year=2017
end_year=2018

yr=start_year
while (yr<=end_year):
    YEARS='['+str(yr)+':'+str(yr)+']'

    for num in range(10):
        gid=griddf['grid_id'][num]
        print(gid,griddf['name'][num])
        
        q = f"""search publications where year in {YEARS} and research_orgs="%s" return publications"""%(gid)
        print(q)
        m1_pubs_api_data = dsl.query_iterative(q, limit=300)
        print("here")
        # simplify dict structure nesting PS careful about running this twice!
        for x in m1_pubs_api_data.publications:
            if 'author_affiliations' in x:
                x['author_affiliations'] = x['author_affiliations'][0]
            else:
                x['author_affiliations'] = []
            
        m1_authors = json_normalize(m1_pubs_api_data.publications, record_path=['author_affiliations'], meta=["id"], errors='ignore')
        #m1_authors.head()
    
        m1_affiliations = json_normalize(json.loads(m1_authors.to_json(orient='records')), record_path=['affiliations'], 
                meta=['id', 'researcher_id', 'first_name', 'last_name'], record_prefix='aff_')
            #m1_affiliations.head()
    
        m1_researchers_1=pd.DataFrame({'researcher_id': m1_affiliations[m1_affiliations['aff_id'] == gid].researcher_id, 'first_name': m1_affiliations[m1_affiliations['aff_id'] == gid].first_name,'last_name': m1_affiliations[m1_affiliations['aff_id'] == gid].last_name,'aff_id': m1_affiliations[m1_affiliations['aff_id'] == gid].aff_id })
        m1_researchers_2=pd.DataFrame({'researcher_id': m1_authors[m1_authors['current_organization_id'] == gid].researcher_id, 'first_name': m1_authors[m1_authors['current_organization_id'] == gid].first_name,'last_name': m1_authors[m1_authors['current_organization_id'] == gid].last_name,'aff_id': m1_authors[m1_authors['current_organization_id'] == gid].current_organization_id })
        m1_researchers = pd.concat([m1_researchers_1, m1_researchers_2])
    
        # Drop empty values and duplicates
        m1_researchers = m1_researchers.replace('', np.nan, inplace=False).dropna().drop_duplicates()
        m1_researchers=m1_researchers.reset_index(drop=True)
    
        fname="m1_researchers_"+gid+"_"+str(yr)+".csv"
        m1_researchers.to_csv(fname, header=True)
        print('saved '+str(len(m1_researchers))+' unique researchers_id in '+fname)
    
        
    yr+=1
    

grid.258799.8 Kyoto University
search publications where year in [2017:2017] and research_orgs="grid.258799.8" return publications
300 / 8691
600 / 8691
900 / 8691
1200 / 8691
1500 / 8691
1800 / 8691
2100 / 8691
2400 / 8691
2700 / 8691
3000 / 8691
3300 / 8691
3600 / 8691
3900 / 8691
4200 / 8691
4500 / 8691
4800 / 8691
5100 / 8691
5400 / 8691
5700 / 8691
6000 / 8691
6300 / 8691
6600 / 8691
6900 / 8691
7200 / 8691
7500 / 8691
7800 / 8691
8100 / 8691
8400 / 8691
8691 / 8691
here


NameError: name 'np' is not defined

In [ ]:
%dslloop search publications where year in [2017:2017] and research_orgs="grid.258799.8" return publications